Vamos a realizar una tarea de clasificacion con una CNN completa, usando MNIST como dataset, el "hellow world" de CV

### Carga del Dataset

In [9]:
# Arranque: imports, dataset y primer batch (MNIST)
import torch, torch.nn as nn, torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


torch.manual_seed(3)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Aqui convertimos las imagenes a tensores
# Es decir, el valor de los pixeles pasa a estar entre 0 y 1
# y el shape de las imagenes pasa a ser (C,H,W) (Canales, Alto, Ancho)

transform = transforms.ToTensor()  # [0,1], shape (C,H,W)

train = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

loader_train = DataLoader(train, batch_size=4, shuffle=True)
loader_test = DataLoader(test, batch_size=4, shuffle=True)


images, labels = next(iter(loader_train))
print(images.shape, labels.shape)





cuda
torch.Size([4, 1, 28, 28]) torch.Size([4])


In [ ]:
#Comprobamos que el shape de las imagenes es el esperado
assert images.ndim == 4
assert images.shape[1] == 1
assert images.shape[2:] == (28, 28)
assert images.min() >= 0.0 and images.max() <= 1.

#### Definición de arquitectura CNN


In [ ]:
import torch
import torch.nn as nn

def get_flatten_size(model_features, input_shape=(1, 1, 28, 28)):
    with torch.no_grad():
        x = torch.zeros(input_shape)
        out = model_features(x)
        return out.view(out.size(0), -1).size(1)

class MiniCNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.features = nn.Sequential(

            #Primera capa de convolucion, entra una imagen en blanco y negro (1 canal) y sale una imagen de 32 features (filtros)
            #El kernel es de 3x3 y el padding es 1 (same padding)
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(), #Aplicamos la funcion de activacion ReLU
            nn.MaxPool2d(kernel_size=2, stride=2), #Aplicamos pooling max para reducir la dimensionalidad de la imagen

            #Segunda capa de convolucion, entra una imagen de 32 features y sale una imagen de 64 features
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(), #Aplicamos la funcion de activacion ReLU
            nn.MaxPool2d(kernel_size=2, stride=2) #Aplicamos pooling max para reducir la dimensionalidad de la imagen
        )

        n_flat = get_flatten_size(self.features)

        

        #Añadimos una capa lineal para clasificar
        self.classifier = nn.Sequential(
            nn.Flatten(), #Aplanamos la imagen para que sea un vector
            nn.Dropout(p=0.3), #Aplicamos dropout para evitar el overfitting
            
            nn.Linear(in_features=n_flat, out_features=64), #Una capa lineal con n_flat entradas y 64 salidas
            nn.ReLU(), #Aplicamos la funcion de activacion ReLU
            
            nn.Dropout(p=0.5), #Aplicamos dropout para evitar el overfitting
            
            nn.Linear(in_features=64, out_features=10), #Una capa lineal con 64 entradas y 10 salidas
            # No aplicamos softmax, ya que la funcion de perdida que usamos (CrossEntropyLoss) lo aplica por defecto
            # nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x





# Instancia
model = MiniCNN()
print(model)

x = torch.randn(4, 1, 28, 28)
logits = model(x)
print(logits.shape)  # esperado: (4, 10)


MiniCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=10, bias=True)
  )
)
torch.Size([4, 10])


Vamos a imprimir un summary

PyTorch usa formato NCHW:

| Posición | Significado                         | Ejemplo |
| -------- | ----------------------------------- | ------- |
| 0        | N → número de imágenes (batch size) | 4   (-1 = None (no definido)    |
| 1        | C → canales o *features maps* (filtros)       | 32      |
| 2        | H → alto de la imagen               | 28      |
| 3        | W → ancho de la imagen              | 28      |


In [19]:
from torchsummary import summary

summary(model.to(device), (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
              ReLU-5           [-1, 64, 14, 14]               0
         MaxPool2d-6             [-1, 64, 7, 7]               0
           Flatten-7                 [-1, 3136]               0
            Linear-8                   [-1, 64]         200,768
              ReLU-9                   [-1, 64]               0
           Linear-10                   [-1, 10]             650
Total params: 220,234
Trainable params: 220,234
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.67
Params size (MB): 0.84
Estimated T